# Importing necessary libraries

In [15]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import to_categorical
from keras import backend as K
from sklearn.model_selection import train_test_split, StratifiedKFold
from keras.models import load_model
import numpy as np
import pandas as pd
import glob
import cv2
import csv

# Define two functions

getFish: takes in a filepath and reads image data using OpenCV. Resizes image to 150x150 in RGB. Returns the image pixel data, the classifications (eyespot, no eyespot), and the species name.  

load_kfold_data: takes in k number of folds and a list of image filespaths. Calls getFish to receive image data and appends image data and classification data to respective lists. Label data is converted to binary in order to run in function StratifiedKFold. Convert labels back to categorical. Returns folds, training data, and labels.

In [2]:
def getFish(filepath):
    img = cv2.imread(filepath)
    # standardize size for prediction step
    img = cv2.resize(img, (150, 150))
    img = img/255.0
    img = img.reshape((1,) + img.shape)
    
    classification, _, species = filepath.partition('_')
    
    #res = model.predict(img)
    return(img, classification, species)

def load_data_kfold(k, image_filepaths):
    
    #use getFish function 
    #load in one file, append to a list
    
    x_train = []
    y_train = []
    
    for i in range(len(image_filepaths)):
        x, y, _ = getFish(image_filepaths[i])
        x_train.append(x)
        y_train.append(y)
        if (y_train[i] == 'eyespot'):
            y_train[i] = 0
        else:
            y_train[i] = 1
    
    #StratifiedKFold can only work in binary; convert labels to categorical afterwards
    folds = list(StratifiedKFold(n_splits=k, shuffle=True, random_state=1).split(x_train, y_train))
    
    y_train = to_categorical(y_train)
    
    return folds, x_train, y_train

# Running load_data_kfold

Load list of image filepaths into "images", and then use kfold function to get 10 folds, the training data, and the trianing labels.

In [3]:
images = glob.glob('*.jpg')

k = 10
folds, x_train, y_train = load_data_kfold(k, images)

# Defining CNN

In [5]:
# dimensions of our images.
img_width, img_height = 150, 150

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('sigmoid'))

model.compile(loss='mean_squared_error',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Creating distorted images generator to create more training images

In [6]:
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1,
                         zoom_range = 0.1,
                         rotation_range = 10
                        )

# Running training set using 10-fold cross validation

Now using the training data to train the model that was previously defined with a batch size of 20 and 30 epochs per fold

In [9]:
batch_size = 20

for j, (train_idx, val_idx) in enumerate(folds):
    
    X_train_cv = []
    y_train_cv = []
    X_valid_cv = []
    y_valid_cv = []
    
    for i in range(len(train_idx)):
        X_train_cv.append(x_train[train_idx[i]])
        y_train_cv.append(y_train[train_idx[i]])
    for i in range(len(val_idx)):
        X_valid_cv.append(x_train[val_idx[i]])
        y_valid_cv.append(y_train[val_idx[i]])
    
    X_train_cv = np.array(X_train_cv)
    X_train_cv = np.squeeze(X_train_cv, axis = 1)
    
    X_valid_cv = np.array(X_valid_cv)
    X_valid_cv = np.squeeze(X_valid_cv, axis = 1)
    
    y_train_cv = np.array(y_train_cv)
    y_valid_cv = np.array(y_valid_cv)
    
    print '\nFold ',j
    name_weights = "final_model_fold" + str(j) + "_weights.h5"
    generator = gen.flow(X_train_cv, y_train_cv, batch_size = batch_size)
    model.fit_generator(
                generator,
                steps_per_epoch=len(X_train_cv)/batch_size,
                epochs=30,
                shuffle=True,
                verbose=1,
                validation_data = (X_valid_cv, y_valid_cv))
    model.save_weights('first_try.h5')
    model.save('second_try.h5')
    print(model.evaluate(X_valid_cv, y_valid_cv))


Fold  0
Epoch 1/30
3/3 [==============================] - 4s 1s/step - loss: 0.2779 - acc: 0.4744 - val_loss: 0.2484 - val_acc: 0.5000
Epoch 2/30
3/3 [==============================] - 2s 803ms/step - loss: 0.2525 - acc: 0.5250 - val_loss: 0.2512 - val_acc: 0.5000
Epoch 3/30
3/3 [==============================] - 3s 907ms/step - loss: 0.2560 - acc: 0.4500 - val_loss: 0.2521 - val_acc: 0.5000
Epoch 4/30
3/3 [==============================] - 2s 798ms/step - loss: 0.2493 - acc: 0.4487 - val_loss: 0.2520 - val_acc: 0.5000
Epoch 5/30
3/3 [==============================] - 2s 782ms/step - loss: 0.2510 - acc: 0.4917 - val_loss: 0.2506 - val_acc: 0.5000
Epoch 6/30
3/3 [==============================] - 3s 922ms/step - loss: 0.2492 - acc: 0.5000 - val_loss: 0.2490 - val_acc: 0.5000
Epoch 7/30
3/3 [==============================] - 3s 847ms/step - loss: 0.2549 - acc: 0.5179 - val_loss: 0.2540 - val_acc: 0.5000
Epoch 8/30
3/3 [==============================] - 3s 960ms/step - loss: 0.2505 - acc

Epoch 3/30
3/3 [==============================] - 3s 951ms/step - loss: 0.2260 - acc: 0.6122 - val_loss: 0.1695 - val_acc: 0.8750
Epoch 4/30
3/3 [==============================] - 3s 1s/step - loss: 0.2258 - acc: 0.5526 - val_loss: 0.2244 - val_acc: 0.5000
Epoch 5/30
3/3 [==============================] - 3s 945ms/step - loss: 0.2097 - acc: 0.6288 - val_loss: 0.1786 - val_acc: 0.6250
Epoch 6/30
3/3 [==============================] - 4s 1s/step - loss: 0.2284 - acc: 0.6000 - val_loss: 0.1789 - val_acc: 0.7500
Epoch 7/30
3/3 [==============================] - 3s 1s/step - loss: 0.2238 - acc: 0.6724 - val_loss: 0.1969 - val_acc: 0.8750
Epoch 8/30
3/3 [==============================] - 3s 1s/step - loss: 0.2419 - acc: 0.5865 - val_loss: 0.2107 - val_acc: 0.7500
Epoch 9/30
3/3 [==============================] - 3s 1s/step - loss: 0.2349 - acc: 0.6462 - val_loss: 0.2188 - val_acc: 0.7500
Epoch 10/30
3/3 [==============================] - 3s 1s/step - loss: 0.2025 - acc: 0.7673 - val_loss: 0.

Epoch 5/30
3/3 [==============================] - 3s 985ms/step - loss: 0.2167 - acc: 0.6635 - val_loss: 0.1083 - val_acc: 0.8750
Epoch 6/30
3/3 [==============================] - 3s 986ms/step - loss: 0.1718 - acc: 0.7673 - val_loss: 0.1433 - val_acc: 0.7500
Epoch 7/30
3/3 [==============================] - 3s 878ms/step - loss: 0.1700 - acc: 0.8096 - val_loss: 0.1262 - val_acc: 0.8750
Epoch 8/30
3/3 [==============================] - 3s 996ms/step - loss: 0.1814 - acc: 0.7667 - val_loss: 0.1336 - val_acc: 0.7500
Epoch 9/30
3/3 [==============================] - 3s 1s/step - loss: 0.1936 - acc: 0.6667 - val_loss: 0.1375 - val_acc: 0.8750
Epoch 10/30
3/3 [==============================] - 4s 1s/step - loss: 0.2026 - acc: 0.6808 - val_loss: 0.1544 - val_acc: 0.7500
Epoch 11/30
3/3 [==============================] - 3s 973ms/step - loss: 0.1737 - acc: 0.7673 - val_loss: 0.1316 - val_acc: 0.7500
Epoch 12/30
3/3 [==============================] - 3s 1s/step - loss: 0.1853 - acc: 0.7417 - v

Epoch 7/30
3/3 [==============================] - 2s 770ms/step - loss: 0.1049 - acc: 0.8449 - val_loss: 0.1807 - val_acc: 0.7500
Epoch 8/30
3/3 [==============================] - 2s 751ms/step - loss: 0.1438 - acc: 0.7929 - val_loss: 0.1817 - val_acc: 0.7500
Epoch 9/30
3/3 [==============================] - 2s 802ms/step - loss: 0.1406 - acc: 0.7833 - val_loss: 0.1758 - val_acc: 0.7500
Epoch 10/30
3/3 [==============================] - 2s 773ms/step - loss: 0.1281 - acc: 0.8186 - val_loss: 0.1559 - val_acc: 0.7500
Epoch 11/30
3/3 [==============================] - 2s 766ms/step - loss: 0.0987 - acc: 0.8878 - val_loss: 0.1685 - val_acc: 0.8750
Epoch 12/30
3/3 [==============================] - 2s 740ms/step - loss: 0.1593 - acc: 0.7500 - val_loss: 0.3763 - val_acc: 0.3750
Epoch 13/30
3/3 [==============================] - 2s 715ms/step - loss: 0.2018 - acc: 0.7423 - val_loss: 0.3015 - val_acc: 0.5000
Epoch 14/30
3/3 [==============================] - 2s 767ms/step - loss: 0.0914 - acc:

3/3 [==============================] - 3s 925ms/step - loss: 0.1128 - acc: 0.8333 - val_loss: 0.0061 - val_acc: 1.0000
Epoch 9/30
3/3 [==============================] - 2s 760ms/step - loss: 0.1080 - acc: 0.8788 - val_loss: 0.0142 - val_acc: 1.0000
Epoch 10/30
3/3 [==============================] - 3s 861ms/step - loss: 0.1129 - acc: 0.8333 - val_loss: 0.0133 - val_acc: 1.0000
Epoch 11/30
3/3 [==============================] - 2s 782ms/step - loss: 0.1013 - acc: 0.8449 - val_loss: 0.2676 - val_acc: 0.6250
Epoch 12/30
3/3 [==============================] - 2s 769ms/step - loss: 0.1429 - acc: 0.8019 - val_loss: 0.0434 - val_acc: 0.8750
Epoch 13/30
3/3 [==============================] - 2s 733ms/step - loss: 0.1092 - acc: 0.8192 - val_loss: 0.0308 - val_acc: 1.0000
Epoch 14/30
3/3 [==============================] - 2s 788ms/step - loss: 0.0937 - acc: 0.8795 - val_loss: 0.0102 - val_acc: 1.0000
Epoch 15/30
3/3 [==============================] - 2s 779ms/step - loss: 0.0806 - acc: 0.8538 - 

# Predicting

Define prediction function, takes in a model and a list of returned data from getFish function. Returns the model's prediction, the correct classification, the prediction weights, and the fish family/species.

In [13]:
def predict(model, img):
    prediction_weights = model.predict(img[0])
    prediction = model.predict_classes(img[0])
    if (prediction[0] == 0):
        prediction = "eyespot"
    else:
        prediction = "noeyespot"
    fam = img[2]
    correct_class = img[1]
    #print "Model classification:",prediction,"\nCorrect classification:", correct_class,"\nPrediction wieghts:",prediction_weights,"\nType:",fam,"\n"
    return(prediction, correct_class, prediction_weights, fam)

# Writing the model's classification data to CSV

Loads model in "mod"  
Writes the model classification, correct classification, prediction weights, and species/family to a csv files  
Calculates data for confusion matrix

In [14]:
mod = load_model("second_try.h5")

with open('classification_data.csv', 'wb') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['Model Classification', 'Correct Classification', 'Prediction Weights', 'Type'])

    true_eye=0
    true_noeye=0
    false_eye=0
    false_noeye=0

    for i in range(len(images)):
        info = predict(mod, getFish(images[i]))
        filewriter.writerow([info[0], info[1], info[2], info[3]])

        #computing info for confusion matrix
        if (info[0] == info[1]):
            if (info[0] == 'eyespot'):
                true_eye += 1
            else:
                true_noeye += 1
        else:
            if (info[0] == 'eyespot'):
                false_eye += 1
            else:
                false_noeye += 1

# Prints confusion matrix

In [16]:
data = {'Eyespots':[true_eye, false_noeye], 'No Eyespots':[false_eye, true_noeye] }
confusion_matrix = pd.DataFrame(data = data)
confusion_matrix = confusion_matrix.rename({0:'Eyespots (Model Output)', 1:'No Eyespots (Model Output)'},axis='index')
confusion_matrix

,Eyespots,No Eyespots
Eyespots (Model Output),39,8
No Eyespots (Model Output),1,32
